## Минимальный необходимый размер выборки

In [1]:
var_control = 2000 # стандартное отклонение контрольной группы
var_test = 2000    # стандартное отклонение тестовой группы
za_2 = -1.96 # Квантинь стандартного норм. распределения, соответствующий половине вероятности ошибки первого рода
zb = -0.842  # 0.2-квантиль стандартного нормального распределения
mde = 60     # Минимальный размер эффекта, который мы хотим зафиксировать в рамках AB теста. Здесь 60 рублей

#минимальный размер выборки
n = ((var_control**2 + var_test**2) * (za_2 + zb)**2)/mde**2
n = round(n)
print(f'n: {n}')

n: 17447


In [7]:
var_historical = 2000
za_2 = -1.96
zb = -0.842
mde = 60
n = ((2*var_historical) * (za_2 + zb)**2)/mde**2
print(f'n: {n}')

n: 8.723559999999999


In [30]:
p = 0.05
var_bernoulli = p*(1-p)
var_bernoulli

0.0475